In [ ]:
#importing necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from scipy.stats import pearsonr
from joblib import Parallel, delayed
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss, confusion_matrix, classification_report
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier


In [2]:
# Import the oversampled dataset
train = pd.read_excel('train_notsampled.xlsx')

# Check the shape and class distribution
print("Loaded data shape:", train.shape)
print("Class distribution:\n", train['TARGET'].value_counts())

Loaded data shape: (95617, 46)
Class distribution:
 TARGET
0    73640
1    21977
Name: count, dtype: int64


In [3]:
train.head()

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,TARGET,credit_score_missing,Term_Long Term,Term_Short Term,Years in current job_1 year,Years in current job_10+ years,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_Home Mortgage,Home Ownership_Own Home,Home Ownership_Rent,Purpose_Business Loan,Purpose_Buy House,Purpose_Buy a Car,Purpose_Debt Consolidation,Purpose_Educational Expenses,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,101332,713,3111440.0,39152.16,11.2,6,0,174895,328768,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,224532,600,1174238.0,13751.06,17.7,10,0,22268,314336,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,452892,750,2700736.0,20953.20,13.0,11,0,292980,764060,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,46332,600,1174238.0,18021.69,33.4,16,0,496014,641740,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,185328,723,914546.0,12575.15,11.7,8,0,29108,200640,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
test = pd.read_csv('test_set.csv')

In [5]:
test.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,fc5bcf5b-35c3-4d1c-8f40-e71c92b3d17f,06b19e70-191a-4620-9ae8-6bb27e910528,Fully Paid,130174,Short Term,NaN,NaN,2 years,Own Home,Debt Consolidation,5396.00,10.5,18.0,11,0,80864,199650.0,0.0,0.0
1,260e82fe-00a0-4ebc-a940-de9700d43fba,8ecda87b-deb9-4376-92fe-7296cbd3f462,Fully Paid,98208,Short Term,723.0,834233.0,3 years,Home Mortgage,Debt Consolidation,21689.83,22.9,46.0,18,0,189506,1160962.0,0.0,0.0
2,ce945b2a-da98-468c-ad7a-9c7738efaadf,37fe9f3e-f4bc-4df8-97ad-58d32470965f,Charged Off,325908,Long Term,707.0,705109.0,2 years,Rent,Debt Consolidation,16511.19,15.4,NaN,8,1,86279,127270.0,1.0,0.0
3,fbd3da73-78e2-49c8-a822-985b21eea849,e6bf52d6-ec07-48a2-b129-4f423eff1ba5,Fully Paid,215754,Short Term,NaN,NaN,4 years,Home Mortgage,Debt Consolidation,20918.81,10.9,NaN,9,1,50217,89474.0,1.0,0.0
4,26386194-7c88-49e3-b5f2-a4cb652ad3d8,a21a1ae7-0115-4b43-af4b-e82b7913685f,Fully Paid,198638,Long Term,686.0,764579.0,3 years,Rent,Debt Consolidation,15533.83,18.1,32.0,10,0,94734,280500.0,0.0,0.0


In [6]:
test.drop(labels=['Loan ID', 'Customer ID'], axis=1, inplace=True)

# These two features are only for identification.Hence droping them

In [7]:
# # Missing Values

# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()

        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)

        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")

        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [8]:
missing_values_table(test)

# A curious thing about the table below is the last 10 features have the same number o missing values.
# I will go deeper and figure out what is happening.

Your selected dataframe has 17 columns.
There are 7 columns that have missing values.


,Missing Values,% of Total Values
Months since last delinquent,2669,53.4
Annual Income,932,18.6
Credit Score,932,18.6
Years in current job,213,4.3
Bankruptcies,18,0.4
Maximum Open Credit,2,0.0
Tax Liens,2,0.0


In [9]:
# Drop the columns with > 50% missing
test.drop(columns = 'Months since last delinquent', axis=1, inplace=True)

In [10]:
# test.drop(test.tail(514).index, inplace=True) # drop last 514 rows
missing_values_table(test)

Your selected dataframe has 16 columns.
There are 6 columns that have missing values.


,Missing Values,% of Total Values
Credit Score,932,18.6
Annual Income,932,18.6
Years in current job,213,4.3
Bankruptcies,18,0.4
Maximum Open Credit,2,0.0
Tax Liens,2,0.0


In [11]:
# As the number of missing values is so low in the 'Maximum Open Credit' I will drop them.

for i in test['Maximum Open Credit'][test['Maximum Open Credit'].isnull() == True].index:
    test.drop(labels=i, inplace=True)
missing_values_table(test)

Your selected dataframe has 16 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
Credit Score,931,18.6
Annual Income,931,18.6
Years in current job,213,4.3
Bankruptcies,18,0.4
Tax Liens,2,0.0


In [12]:
# As the number of missing values is so low in the 'Tax Liens' I will drop them.

for i in test['Tax Liens'][test['Tax Liens'].isnull() == True].index:
    test.drop(labels=i, inplace=True)
missing_values_table(test)

Your selected dataframe has 16 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
Credit Score,930,18.6
Annual Income,930,18.6
Years in current job,213,4.3
Bankruptcies,16,0.3


In [13]:
# As the number of missing values is so low in the 'Bankruptcies' I will drop them.

for i in test['Bankruptcies'][test['Bankruptcies'].isnull() == True].index:
    test.drop(labels=i, inplace=True)
missing_values_table(test)

Your selected dataframe has 16 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
Credit Score,929,18.7
Annual Income,929,18.7
Years in current job,213,4.3


In [14]:
test.head()

,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,Fully Paid,130174,Short Term,NaN,NaN,2 years,Own Home,Debt Consolidation,5396.00,10.5,11,0,80864,199650.0,0.0,0.0
1,Fully Paid,98208,Short Term,723.0,834233.0,3 years,Home Mortgage,Debt Consolidation,21689.83,22.9,18,0,189506,1160962.0,0.0,0.0
2,Charged Off,325908,Long Term,707.0,705109.0,2 years,Rent,Debt Consolidation,16511.19,15.4,8,1,86279,127270.0,1.0,0.0
3,Fully Paid,215754,Short Term,NaN,NaN,4 years,Home Mortgage,Debt Consolidation,20918.81,10.9,9,1,50217,89474.0,1.0,0.0
4,Fully Paid,198638,Long Term,686.0,764579.0,3 years,Rent,Debt Consolidation,15533.83,18.1,10,0,94734,280500.0,0.0,0.0


In [15]:
# Map values in 'loan_status' column /target column to numerical
test['TARGET'] = test['Loan Status'].map({
    'Fully Paid': 0, #non- Defaulter
    'Charged Off': 1 # defaulter
})


In [16]:
test = test.drop('Loan Status', axis=1)

In [17]:
test['credit_score_missing'] = test['Credit Score'].isna().astype(int)


In [18]:
test.head()

,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,TARGET,credit_score_missing
0,130174,Short Term,NaN,NaN,2 years,Own Home,Debt Consolidation,5396.00,10.5,11,0,80864,199650.0,0.0,0.0,0,1
1,98208,Short Term,723.0,834233.0,3 years,Home Mortgage,Debt Consolidation,21689.83,22.9,18,0,189506,1160962.0,0.0,0.0,0,0
2,325908,Long Term,707.0,705109.0,2 years,Rent,Debt Consolidation,16511.19,15.4,8,1,86279,127270.0,1.0,0.0,1,0
3,215754,Short Term,NaN,NaN,4 years,Home Mortgage,Debt Consolidation,20918.81,10.9,9,1,50217,89474.0,1.0,0.0,0,1
4,198638,Long Term,686.0,764579.0,3 years,Rent,Debt Consolidation,15533.83,18.1,10,0,94734,280500.0,0.0,0.0,0,0


In [19]:
# Check average target value when credit score is missing vs not missing
print(test.groupby('credit_score_missing')['TARGET'].mean())


credit_score_missing
0    0.210318
1    0.294941
Name: TARGET, dtype: float64


In [20]:
test['Credit Score'] = test['Credit Score'].fillna(600)  # A conservative, lower-end score


In [21]:
# Calculate the mode of the column
mode_value = test['Years in current job'].mode()[0]

# Fill missing values with the mode
test['Years in current job'].fillna(mode_value, inplace=True)


C:\Users\easntop\AppData\Local\Temp\ipykernel_31480\3908443526.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Years in current job'].fillna(mode_value, inplace=True)


In [22]:
# Calculate the median of Annual Income
median_income = test['Annual Income'].median()

# Fill missing values with the median
test['Annual Income'].fillna(median_income, inplace=True)



C:\Users\easntop\AppData\Local\Temp\ipykernel_31480\3716808664.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Annual Income'].fillna(median_income, inplace=True)


In [23]:
for col in ['Credit Score', 'Annual Income']:
    q1 = np.percentile(train[col], 1)
    q99 = np.percentile(train[col], 99)
    train[col] = np.clip(train[col], q1, q99)  # modifies in-place as you want


In [24]:
# Assume your DataFrame is called 'df'
categorical_columns = test.select_dtypes(include=['object', 'category']).columns
num_categorical = len(categorical_columns)

print(f"Number of categorical features: {num_categorical}")
print("Categorical columns:")
print(categorical_columns.tolist())


Number of categorical features: 4
Categorical columns:
['Term', 'Years in current job', 'Home Ownership', 'Purpose']


In [25]:
# Select the categorical columns
categorical_cols = ['Term', 'Years in current job', 'Home Ownership', 'Purpose']

# One-hot encode the categorical columns (only those columns, not the full DataFrame)
categorical_encoded = pd.get_dummies(test[categorical_cols], drop_first=False, dtype=int)

# Drop the original categorical columns from the DataFrame
test.drop(columns=categorical_cols, inplace=True)

# Concatenate the encoded columns back to the original DataFrame
test = pd.concat([test, categorical_encoded], axis=1)


In [26]:
test.head()

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,TARGET,credit_score_missing,Term_Long Term,Term_Short Term,Years in current job_1 year,Years in current job_10+ years,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_Home Mortgage,Home Ownership_Own Home,Home Ownership_Rent,Purpose_Business Loan,Purpose_Buy House,Purpose_Buy a Car,Purpose_Debt Consolidation,Purpose_Educational Expenses,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,130174,600.0,1167816.0,5396.00,10.5,11,0,80864,199650.0,0.0,0.0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,98208,723.0,834233.0,21689.83,22.9,18,0,189506,1160962.0,0.0,0.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,325908,707.0,705109.0,16511.19,15.4,8,1,86279,127270.0,1.0,0.0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,215754,600.0,1167816.0,20918.81,10.9,9,1,50217,89474.0,1.0,0.0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,198638,686.0,764579.0,15533.83,18.1,10,0,94734,280500.0,0.0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
X_test = test.drop(columns=['TARGET'])
y_test = test['TARGET']

In [28]:
X = train.drop('TARGET', axis=1)   
y = train['TARGET']

In [ ]:
# -------------------------
#  Gini Impurity
# -------------------------
def gini_impurity(y):
    counts = np.bincount(y)                           # counts how many times each class label appears.
    prob_sq_sum = np.sum((counts / len(y)) ** 2)      # probability squared sum
    return 1 - prob_sq_sum

# -------------------------
# Feature Importance 
# -------------------------
def compute_feature_importance(X, y):
    importances = []                                            # A python list list[floats]
    for i in range(X.shape[1]):                                 # loop over Number of column/features in dataset 
        try:                                                    # try catches Pearson error(constant values,Nan Values etc)
            corr, _ = pearsonr(X[:, i], y)                      # pearsonr-> function returns correlation coeff & X[:,i] -> Selects all rows and column i/feature i ,y -> Target
            importances.append(abs(corr))                       # Add the absolute value of corr to the list -> importance as corr -Correlation Coefficent can be a value between -1 and 1
        except Exception: 
            importances.append(0.0)                             # if any exception happens the importance is set 0 for that particular feature
    importances = np.nan_to_num(importances)
    return importances / (np.sum(importances) + 1e-8)           # normalizing the feature importance values.1e-8 is used so that denominator doesnt compeletly becomes zero

# importances = np.nan_to_num(importances) Pearson returns NaN silently for constant columns — no error raised — so the except block is never hit.
#                                           may later divide or sum NaNs — results in errors
# (np.sum(importances) + 1e-8)-> normalizes -> To scale all importances between 0 and 1
#                                            -> To ensure total importance sums to 1 (prob-like)

# -------------------------
# Single Quantile Extra Tree (with max_features)
# -------------------------
class ExtraTreeQuantileClassifier:
    def __init__(self, max_depth=None, min_samples_split=2, feature_quantiles=None, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.feature_quantiles = feature_quantiles
        self.max_features = max_features
        self.tree = None                                    # store the built tree structure (after .fit())

    def fit(self, X, y):    # starts looking at the train data to build tree
        self.n_classes_ = len(set(y))                       # Number of classes /TARGET
        self.n_features_ = X.shape[1]                       # Number of features
        self.tree = self._build_tree(X, y, depth=0)         # Start building the tree (recursive function) starting from root node (depth=0).

    def _build_tree(self, X, y, depth): #a recursive function to build tree
        num_samples, num_features = X.shape

        # Stopping conditions --->>>> # Stop splitting if depth or sample size or purity condition met otherwise split
        #This checks when to stop splitting and make a leaf node.
        if (self.max_depth is not None and depth >= self.max_depth) or \
           num_samples < self.min_samples_split or len(set(y)) == 1:
            most_common = Counter(y).most_common(1)[0][0] #If all samples belong to the same class, no need to split
            return {'type': 'leaf', 'class': most_common}

        # Choose random subset of features (max_features)   
        #  ""  block controls feature randomness (feature bagging) "" 
        # """ Out of all features, which subset of features should I consider for splitting?  """

        if self.max_features is None:                # use all features available
            features_to_consider = np.arange(num_features) 
        elif isinstance(self.max_features, int):      # if max_features is an integer (like 3):randomly chosen 3 features out of 5
            features_to_consider = np.random.choice(num_features, self.max_features, replace=False)
        elif isinstance(self.max_features, float):    # If max_features is a float 
            n_feats = max(1, int(self.max_features * num_features))
            features_to_consider = np.random.choice(num_features, n_feats, replace=False)
        elif self.max_features == "sqrt":             ## If max_features is a float (like 9) : take 3
            n_feats = max(1, int(np.sqrt(num_features)))
            features_to_consider = np.random.choice(num_features, n_feats, replace=False)
        else:
            raise ValueError("Invalid max_features value")

        # Find best split over these features
        best_gini = 1.0                     # (maximum impurity, worst possible) Lower Gini = better split (purer node).
        best_split = None                   # No split chosen yet (None)

        for feature_idx in features_to_consider:
            n_quantiles = self.feature_quantiles.get(feature_idx, 10) #If that feature is not present in the dictionary (maybe due to some missing calculation or initialization issue), then default to 10 quantiles.
                                                                     # retrieves how many quantiles to consider for this feature.For the current feature only, it stores how many quantiles to use to find candidate split points.
            unique_vals = np.unique(X[:, feature_idx])               # extracts all distinct (unique) values present in that feature across the bootstrapped data.
            if len(unique_vals) == 1:                          #If only 1 unique value exists, then this feature cannot be split — because there's nothing to divide.
                continue                                       #So the code skips this feature (continue) and moves to the next feature.

            thresholds = np.unique(np.quantile(X[:, feature_idx],
                                               np.linspace(0, 1, n_quantiles + 2)[1:-1])) # "Find possible threshold points to split this feature at, using quantiles.

            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold           # has values of feature less than threshold ,datatype of left_mask is boolean numpy array
                right_mask = X[:, feature_idx] > threshold           # has values of feature more than threshold
                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue                      #If **either left or right node gets zero samples — i.e. no data fell on that side of the split — then this threshold is invalid. go to next

                y_left, y_right = y[left_mask], y[right_mask]
                gini = (len(y_left) / num_samples) * gini_impurity(y_left) + \
                       (len(y_right) / num_samples) * gini_impurity(y_right)   # calculates total/weighted gini using gini of individual left node and right node gini

                if gini < best_gini:
                    best_gini = gini            # updating best gini if lower gini is found
                    best_split = {               #stores details of best split
                        'feature': feature_idx,
                        'threshold': threshold,
                        'left_indices': left_mask,
                        'right_indices': right_mask
                    }

        if best_split is None:         # "If no good split exists, stop splitting and make this node predict the majority class seen so far."
            most_common = Counter(y).most_common(1)[0][0]  #finds the majority class label of the samples present at this node.
            return {'type': 'leaf', 'class': most_common}

        left_subtree = self._build_tree(X[best_split['left_indices']], y[best_split['left_indices']], depth + 1) # recursive call for left node
        right_subtree = self._build_tree(X[best_split['right_indices']], y[best_split['right_indices']], depth + 1) #  recursive call for right node

        return {   #after each node is constructed return stores the deatils of the each node of tree and returns it in recursive manner
            'type': 'node',
            'feature': best_split['feature'],
            'threshold': best_split['threshold'],
            'left': left_subtree,
            'right': right_subtree
        }

# core function used to predict the class label of a single data sample x 
# by traversing the decision tree from the root to the appropriate leaf node.
    def _predict_sample(self, x, node):
        if node['type'] == 'leaf':
            return node['class']
        if x[node['feature']] <= node['threshold']:
            return self._predict_sample(x, node['left'])
        else:
            return self._predict_sample(x, node['right'])

    def predict(self, X):
        return np.array([self._predict_sample(x, self.tree) for x in X])
     #Predict one sample at a time using _predict_sample()
    #Collect all results into an array of predictions.
    #Used by predict(X_test) to predict entire test sets.


# -------------------------
# Ensemble of Extra Trees (Quantile-based, with n_jobs for parallel)
# -------------------------
class ExtraTreeQuantileEnsembleClassifier:
    def __init__(self, n_estimators=5, max_depth=None, min_samples_split=2,
                 max_quantiles=200, min_quantiles=3, max_features=None, n_jobs=1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_quantiles = max_quantiles
        self.min_quantiles = min_quantiles
        self.max_features = max_features
        self.n_jobs = n_jobs  # Added for multiprocessing
        self.trees = []
        #The quantile_map dictionary, which stores how many quantiles to assign for each feature,is saved in the model’s object (self) as self.feature_quantiles.

    def fit(self, X, y):
        # Calculate feature importance (Pearson) for quantile assignment
        feature_importances = compute_feature_importance(X, y)            # compute_feature_importance returns normalized importance values for each feature in X           
        quantile_map = {}                                                 # an empty dictionary to store how many quantiles each feature will get

        for i, imp in enumerate(feature_importances):                     # Loop over each feature (by index i) and its importance imp.
            quant_count = int(self.min_quantiles + imp * (self.max_quantiles - self.min_quantiles)) # based on its importance calculate number of quantiles (quant_count) to use for the feature 
            quantile_map[i] = max(self.min_quantiles, quant_count)        # max(self.min_quantiles, quant_count) ensures no feature gets less than the minimum allowed quantiles — for safety.

        self.feature_quantiles = quantile_map                        # maps features to quantiles  self.feature_quantiles = {0: 101, 1: 62, 2: 42}

        def build_tree(X, y):
            indices = np.random.choice(len(X), len(X), replace=True) # Draw random sample indices (bootstrapping) (range of values to sample from,values to draw, with replacement)
            X_sample = X[indices]                                    # Give those specific rows from X, in the order given by indices (rows selected in previous step)
            y_sample = y[indices]                                    # Give the target values (labels) for those particular rows stored in x_sample

            tree = ExtraTreeQuantileClassifier(max_depth=self.max_depth,                 # how deep it can go (limit recursion depth)
                                               min_samples_split=self.min_samples_split, # when to stop splitting,
                                               feature_quantiles=self.feature_quantiles, # how many quantiles to use for each feature (calculated earlier)
                                               max_features=self.max_features)           # how many features to consider at each split.
            tree.fit(X_sample, y_sample)       # train (fit) this tree on X_sample, y_sample
            return tree

        # Parallel tree building here - helps reducing predictiontime as it uses all cores
        self.trees = Parallel(n_jobs=self.n_jobs)(
            delayed(build_tree)(X, y) for _ in range(self.n_estimators)
        )

    def predict(self, X):# is only called at prediction time on .predict(X_test)
        predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.apply_along_axis(lambda x: Counter(x).most_common(1)[0][0], axis=0, arr=predictions)
                  #It applies a function to 1D slices (rows or columns) of a multi-dimensional NumPy array.

                  # self.trees is a list of trained individual decision trees in our ensemble.
                  # For each tree in this list, we are calling its own .predict(X) method
                  # "Make each tree predict all samples in X — collect all these predictions in a 2D array — one row per tree."

  
    # Test

if __name__ == "__main__":
    from sklearn.datasets import make_classification
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

    clf = ExtraTreeQuantileEnsembleClassifier(n_estimators=30, max_quantiles=200, min_quantiles=10, n_jobs=-1)  # n_jobs=-1 to use all cores
    clf.fit(X.values, y.values) #Fitting model on train data
    y_pred = clf.predict(X_test.values) #fitting model on test data
    print("Accuracy:", accuracy_score(y_test, y_pred))
    # print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8919678714859438
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.98      0.93      3854
           1       0.88      0.61      0.72      1126

    accuracy                           0.89      4980
   macro avg       0.89      0.79      0.83      4980
weighted avg       0.89      0.89      0.88      4980

